In [1]:
import torch
import time

计算Jacobi Matrix

In [2]:
def G_modified(X, model):
    # 开始计时
    start = time.time()
    
    input_dim, m = model.W.shape  # m: 隐藏层神经元数量, input_dim: 输入维度
    batch_size = X.shape[0]       # batch_size: 批处理大小
    
    # 初始化 Jacobian 矩阵 J，大小为 (batch_size, m * (input_dim + 1))
    J = torch.zeros(batch_size, m * (input_dim + 1), device=X.device)
    
    # 计算所有样本的 <w_i, x> 和 ReLU 激活
    relu_input = X @ model.W  # (batch_size, m)
    relu_output = torch.relu(relu_input)  # (batch_size, m)
    
    # 对 w_i 的部分并行计算 Jacobian
    W_grad = torch.zeros(batch_size, input_dim, m)
    for j in range(m):
        mask = (relu_input[:, j] > 0).float()  # 只选择 ReLU 激活大于0的元素
        W_grad[:, :, j] = model.a[j] * X * mask.view(-1, 1) / m
    J[:, :m*input_dim] = W_grad.reshape(W_grad.shape[0], -1)
    # 对 a_i 的部分并行计算 Jacobian
    J[:, m*input_dim:] = relu_output / m
    end = time.time()
    print("计算Jacobian矩阵耗时：", end - start)
    return J

In [3]:
def G_modified_CUDA(X, model):
    # 开始计时
    start = time.time()
    # 确保所有张量在 CUDA 设备上
    device = X.device
    
    input_dim, m = model.W.shape  # m: 隐藏层神经元数量, input_dim: 输入维度
    batch_size = X.shape[0]       # batch_size: 批处理大小
    
        # 初始化 Jacobian 矩阵 J，大小为 (batch_size, m * (input_dim + 1))
    J = torch.zeros(batch_size, m * (input_dim + 1), device=device)
    
    # 计算所有样本的 <w_i, x> 和 ReLU 激活
    relu_input = X @ model.W  # (batch_size, m)
    relu_output = torch.relu(relu_input)  # (batch_size, m)
    
    # 对 w_i 的部分并行计算 Jacobian
    mask = (relu_input > 0).float()  # (batch_size, m)
    
    # 使用广播机制计算 W_grad
    W_grad = (X.unsqueeze(2) * mask.unsqueeze(1)) / m  # (batch_size, input_dim, m)
    W_grad = W_grad * model.a.view(1, 1, m)  # (batch_size, input_dim, m)
    
    # 将 W_grad 转换为二维矩阵并赋值给 J
    J[:, :m*input_dim] = W_grad.reshape(batch_size, -1)
    
    # 对 a_i 的部分并行计算 Jacobian
    J[:, m*input_dim:] = relu_output / m
    
    end = time.time()
    print("CUDA计算Jacobian矩阵耗时：", end - start)
    return J

In [2]:
class test_model(torch.nn.Module):
    def __init__(self, D, m, W: torch.Tensor, a: torch.Tensor):
        super(test_model, self).__init__()
        self.m = m
        self.W = torch.nn.Parameter(W, requires_grad=True)
        self.a = torch.nn.Parameter(a, requires_grad=True)

    def forward(self, X):
        return torch.nn.ReLU()(X @ self.W) @ self.a / self.m
    
    def loss(self, X, Y):
        y_pred = self.forward(X)
        return (y_pred - Y.reshape(y_pred.shape)) ** 2

In [5]:
D, m = 3, 2
W = torch.tensor([[0.5, -0.3], [0.8, 0.6], [-0.2, 0.7]])
a = torch.tensor([0.9, -1.1])
X = torch.tensor([[1., 2., 3.], [0.5, 1.0, 1.5], [3., -2., 5]])
model = test_model(3, 2, W, a)

In [6]:
# 计算 G_modified
J = G_modified(X, model)
print(J)
J_cuda = G_modified_CUDA(X, model)
print(J_cuda)

计算Jacobian矩阵耗时： 0.006013154983520508
tensor([[ 0.4500, -0.5500,  0.9000, -1.1000,  1.3500, -1.6500,  0.7500,  1.5000],
        [ 0.2250, -0.2750,  0.4500, -0.5500,  0.6750, -0.8250,  0.3750,  0.7500],
        [ 0.0000, -1.6500, -0.0000,  1.1000,  0.0000, -2.7500,  0.0000,  0.7000]],
       grad_fn=<CopySlices>)
CUDA计算Jacobian矩阵耗时： 0.0005838871002197266
tensor([[ 0.4500, -0.5500,  0.9000, -1.1000,  1.3500, -1.6500,  0.7500,  1.5000],
        [ 0.2250, -0.2750,  0.4500, -0.5500,  0.6750, -0.8250,  0.3750,  0.7500],
        [ 0.0000, -1.6500, -0.0000,  1.1000,  0.0000, -2.7500,  0.0000,  0.7000]],
       grad_fn=<CopySlices>)


In [9]:
def auto_grad_G(X, model):
    # length = model.W.shape[0] * model.W.shape[1]
    # height = X.shape[0]
    # J = torch.zeros(height, length)
    # y = model(X).flatten()
    # grad_y = torch.zeros(y.shape)
    # for i in range(y.shape[0]):
    #     grad_y.zero_()
    #     grad_y[i] = 1
    #     w_grad = torch.autograd.grad(y, model.W, grad_y, retain_graph=True, create_graph=True)[0]
    #     J[i] = w_grad.flatten()
    # return J
    output = model(X)
    output.backward()
    return model.W.grad, model.a.grad

model = test_model(3, 2, W, a)
X = torch.tensor([[1.0, 2.0, 3.0]])
w_grad, a_grad = auto_grad_G(X, model)
print(model.W.data.flatten())
print(w_grad)
print(model.W.grad.flatten())
print(model.a.data.flatten(), a_grad.flatten())

tensor([ 0.5000, -0.3000,  0.8000,  0.6000, -0.2000,  0.7000])
tensor([[ 0.4500, -0.5500],
        [ 0.9000, -1.1000],
        [ 1.3500, -1.6500]])
tensor([ 0.4500, -0.5500,  0.9000, -1.1000,  1.3500, -1.6500])
tensor([ 0.9000, -1.1000]) tensor([0.7500, 1.5000])


W,a的Euler梯度

In [11]:
import numpy as np
import random

D, m = 3, 2
W = torch.tensor([[0.5, -0.3], [0.8, 0.6], [-0.2, 0.7]])
a = torch.tensor([0.9, -1.1])
X = torch.tensor([[1., 2., 3.], [0.5, 1.0, 1.5]])
Y = torch.tensor([1., 0.])

In [4]:
model = test_model(D, m, W, a)
print(model(X))
print(model.loss(X, Y))

tensor([-0.9750, -0.4875, -0.7700], grad_fn=<DivBackward0>)
tensor([3.9006, 0.2377, 3.1329], grad_fn=<PowBackward0>)


In [6]:
def N_grad(model, X, Y):
    start = time.time()
    # 获取模型的参数 a 和 W
    with torch.no_grad():
        a, W, hidden_dim = model.a, model.W, model.W.shape[1]
        y_pred = model(X)
        # 计算损失
        grad_a = torch.zeros_like(a)  # 初始化 grad_a
        z1 = torch.nn.ReLU()(torch.mm(X, W))  # 计算 z1 (激活后的隐藏层输出)
        
        # 计算每个隐藏神经元的梯度
        for j in range(hidden_dim):
            grad_a[j] = (2 * (y_pred - Y.reshape(y_pred.shape)) * z1[:, j]).mean() / hidden_dim
        
        # Step 2: 对 W 的梯度
        grad_W = torch.zeros_like(W)  # 初始化 grad_W
        
        # 计算 ReLU 的梯度
        for j in range(hidden_dim):
            for i in range(X.shape[0]):  # 对每个样本进行求和
                if z1[i, j] > 0:  # ReLU 导数 (只有正值才有梯度)
                    grad_W[:, j] += (2 * (y_pred[i] - Y[i]) * a[j] / hidden_dim) * X[i]
        
        grad_W /= X.shape[0]  # 对样本数 N 进行归一化
    end = time.time()
    print("计算梯度耗时：", end - start)
    return grad_W, grad_a

In [7]:
grad_W, grad_a = N_grad(model, X, Y)
print(grad_W)
print(grad_a)

计算梯度耗时： 0.012209177017211914
tensor([[-0.6656,  2.7605],
        [-1.3313,  0.3291],
        [-1.9969,  5.6856]])
tensor([-1.1094, -3.0448])


In [12]:
model = test_model(D, m, W, a)
start = time.time()
loss = model.loss(X, Y).mean()
loss.backward()
end = time.time()
print("自动求导耗时：", end - start)
print(model.W.grad)
print(model.a.grad)
grad_W, grad_a = N_grad(model, X, Y)
print(grad_W)
print(grad_a)

自动求导耗时： 0.0026521682739257812
tensor([[-0.9984,  1.2203],
        [-1.9969,  2.4406],
        [-2.9953,  3.6609]])
tensor([-1.6641, -3.3281])
计算梯度耗时： 0.000743865966796875
tensor([[-0.9984,  1.2203],
        [-1.9969,  2.4406],
        [-2.9953,  3.6609]])
tensor([-1.6641, -3.3281])
